
# Trabajo Practico final NLP

### Integrante: Tomás Rodríguez Griñó

### Legajo: R-4643/4

Importacion de librerias

In [1]:
pip install tensorflow==2.11


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==2.11 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)
ERROR: No matching distribution found for tensorflow==2.11


In [31]:
import chromadb
from langchain.vectorstores import Chroma
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from transformers import AutoTokenizer, AutoModel
import torch
import time

# Ejercicio 1

Lo primero que debemos realizar, es una busqueda de datos para darle contexto a el chatbot

In [71]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Configuración de Selenium
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

try:
    # URL de la página a scrapear
    url = "https://boardgamegeek.com/boardgame/371942/the-white-castle"
    driver.get(url)

    # Extraer la descripción
    wait = WebDriverWait(driver, 60)
    description_div = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "game-description-body")))
    inner_description_div = description_div.find_element(By.CLASS_NAME, "ng-binding")
    description_text = inner_description_div.text
    print("Descripción:", description_text)
finally:
    driver.quit()

Descripción: The heron flies over the Himeji sky while the Daimyo, from the top of the castle, watches his servants move. Gardeners tend the pond, where the koi carp live, warriors stand guard on the walls, and courtiers crowd the gates, pining for an audience that brings them closer to the innermost circles of the court. When night falls, the lanterns are lit and the workers return to their clan.
In The White Castle, players will control one of these clans in order to score more victory points than the rest. To do so, they must amass influence in the court, manage resources boldly, and place their workers in the right place at the right time. The authors are Sheila Santos and Israel Cendrero, the duo known as Llama Dice who also designed the successful The Red Cathedral with Devir. In this case, we leave the Moscow of Ivan the Terrible behind to explore the most imposing fortress in modern Japan, Himeji Castle, where the banner of the Sakai clan flies under the orders of Daimyo Sakai 

Ahora se utiliza web scrapping con Selenium para extraer tanto informacion como consultas y opiniones de usuarios de la pagina Misut Meeple

In [10]:
def scrape_text():
    # URL de la página
    url = "https://misutmeeple.com/2023/11/resena-the-white-castle/"

    # Configuración de Selenium
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)

    try:
        # Navegar a la URL
        driver.get(url)

        # Esperar a que el contenido se cargue
        wait = WebDriverWait(driver, 15)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

        # Extraer texto de <p> y encabezados
        paragraphs = driver.find_elements(By.TAG_NAME, 'p')
        headers = driver.find_elements(By.TAG_NAME, 'h1') + driver.find_elements(By.TAG_NAME, 'h2') + driver.find_elements(By.TAG_NAME, 'h3')

        # Concatenar texto
        all_text = ""
        for element in paragraphs + headers:
            all_text += element.text + "\n"

        return all_text
    finally:
        driver.quit()


text = scrape_text()


# Text splitting

In [12]:
def split_text(text):
    # Configuración del splitter
    splitter = CharacterTextSplitter(separator="\n", chunk_size=500, chunk_overlap=50)
    return splitter.split_text(text)

chunks = split_text(text)


Created a chunk of size 512, which is longer than the specified 500
Created a chunk of size 1090, which is longer than the specified 500
Created a chunk of size 909, which is longer than the specified 500
Created a chunk of size 949, which is longer than the specified 500
Created a chunk of size 643, which is longer than the specified 500
Created a chunk of size 838, which is longer than the specified 500
Created a chunk of size 822, which is longer than the specified 500
Created a chunk of size 738, which is longer than the specified 500
Created a chunk of size 518, which is longer than the specified 500
Created a chunk of size 549, which is longer than the specified 500
Created a chunk of size 508, which is longer than the specified 500
Created a chunk of size 593, which is longer than the specified 500
Created a chunk of size 547, which is longer than the specified 500
Created a chunk of size 584, which is longer than the specified 500
Created a chunk of size 523, which is longer th

# Embedding

In [60]:
from transformers import AutoTokenizer, AutoModel
import torch

# Inicializa el modelo y el tokenizer de Hugging Face
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def embed_documents(texts):
    # Tokeniza y genera los embeddings
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        # Genera los embeddings promediando las representaciones de las últimas capas
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings.numpy()

# Generar los embeddings de los textos
embeddings = embed_documents(chunks)

# Verificar que la longitud de embeddings coincida con la longitud de los textos
assert len(chunks) == len(embeddings), "Las longitudes de los textos y los embeddings no coinciden."

# Asignar un ID único a cada documento
ids_textos = [f"doc{i}" for i in range(1, len(chunks) + 1)]

# Guaradado en ChromaDB

In [61]:
# Configuración inicial de ChromaDB
client = chromadb.Client()

# Crear o acceder a una colección en ChromaDB
collection = client.get_or_create_collection("mis-documentos")

# Guardar los documentos y embeddings en ChromaDB
collection.add(
    documents=chunks,              # Los documentos originales
    metadatas=[{"source": "fuente1"} for _ in chunks],  # Metadata opcional, en este caso solo una fuente
    ids=ids_textos,                # IDs para los documentos
    embeddings=embeddings.tolist() # Los embeddings generados previamente
)

print("Embeddings guardados en ChromaDB con éxito.")

Insert of existing embedding ID: doc1
Insert of existing embedding ID: doc2
Insert of existing embedding ID: doc3
Insert of existing embedding ID: doc4
Add of existing embedding ID: doc1
Add of existing embedding ID: doc2
Add of existing embedding ID: doc3
Add of existing embedding ID: doc4


Embeddings guardados en ChromaDB con éxito.
